# Tutorial: Machine Learning in scikit-learn
*From the [Sebastian Raschka](https://github.com/rasbt/pattern_classification) the GitHub repository [pattern_classification](https://github.com/rasbt/pattern_classification)*

![Machine learning](images/01_robot.png)

<br>
<br>

# Encoding Categorical Features in Classification Tasks

Features can come in various different flavors. Typically we distinguish between

- continuous and
- categorical (discrete)
features.


And the categorical features can be categorized further into:

- ordinal and
- nominal (= no order implied) features.



Now, most implementations of machine learning algorithms require numerical data as input, and we have to prepare our data accordingly. This notebook contains some useful tips for how to encode categorical features using Python pandas and scikit-learn.

<br>
<br>

### Sections

- [Example Data](#Example-Data)
- [Class Labels](#Class-Labels)
- [Ordinal Features](#Ordinal-Features)
- [Nominal Features](#Nominal-Features)
- [Inverse Mapping](#Inverse-Mapping)
- [Using scikit-learn and pandas features](#Using-scikit-learn-and-pandas-features)
    - [scikit LabelEncoder](#scikit-LabelEncoder)
    - [scikit DictVectorizer](#scikit-DictVectorizer)
    - [scikit OneHotEncoder](#scikit-OneHotEncoder)
    - [pandas get_dummies](#pandas-get_dummies)

<br>
<br>

## Example Data

First, let us create a simple example dataset with 3 different kinds of features:
- color: nominal
- size: ordinal
- prize: continuous

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('./data/titanic_curso.csv', sep=',')
df.head()

,pclass,survived,age,embarked,sex
0,1st,KO,29.0000,Southampton,female
1,1st,No KO,2.0000,Southampton,female
2,1st,No KO,30.0000,Southampton,male
3,1st,No KO,25.0000,Southampton,female
4,1st,KO,0.9167,Southampton,male


<br>
<br>

## Class Labels

"Typical" machine learning algorithms handle class labels with "no order implied" - unless we use a ranking classifier (e.g., SVM-rank). Thus, it is save to use a simple set-item-enumeration to convert the class labels from a string representation into integers.

In [2]:
label_mapping = {label:idx for idx,label in enumerate(set(df['survived']))}

df['survived'] = df['survived'].map(label_mapping)
df.head()

,pclass,survived,age,embarked,sex
0,1st,1,29.0000,Southampton,female
1,1st,0,2.0000,Southampton,female
2,1st,0,30.0000,Southampton,male
3,1st,0,25.0000,Southampton,female
4,1st,1,0.9167,Southampton,male


<br>
<br>

## Ordinal Features

Ordinal features need special attention: We have to make sure that the correct values are associated with the corresponding strings. Thus, we need to set-up an explicit mapping dictionary:

In [3]:
df['pclass'].unique()

array(['1st', '2nd', '3rd'], dtype=object)

In [4]:
pclass_mapping = {
           '1st': 2,
           '2nd': 1,
           '3rd': 0}

df['pclass'] = df['pclass'].map(pclass_mapping)
print df['pclass'].unique()
df.head()

[2 1 0]


,pclass,survived,age,embarked,sex
0,2,1,29.0000,Southampton,female
1,2,0,2.0000,Southampton,female
2,2,0,30.0000,Southampton,male
3,2,0,25.0000,Southampton,female
4,2,1,0.9167,Southampton,male


This also can be done with a sklearn preprocessing method call *LabelEncoder*

<br>
<br>

## Nominal Features

Unfortunately, we can't simply apply the same mapping scheme to the `sex` and `embarked` columns that we used for the `pclass`-mapping above. However, we can use another simply trick and convert the "sex" and "ambarked" into binary features: Each possible value becomes a feature column itself (with values 1 or 0).

In [5]:
df['embarked'].unique()

array(['Southampton', 'Cherbourg', 'Queenstown'], dtype=object)

In [6]:
embarked_mapping = {
           'Southampton': (0,0,1),
           'Cherbourg': (0,1,0),
           'Queenstown': (1,0,0)}

df['embarked'] = df['embarked'].map(embarked_mapping)
df.head()

,pclass,survived,age,embarked,sex
0,2,1,29.0000,"(0, 0, 1)",female
1,2,0,2.0000,"(0, 0, 1)",female
2,2,0,30.0000,"(0, 0, 1)",male
3,2,0,25.0000,"(0, 0, 1)",female
4,2,1,0.9167,"(0, 0, 1)",male


In [7]:
sex_mapping = {
           'male': (0,1),
           'female': (1,0)}

df['sex'] = df['sex'].map(sex_mapping)
df.head()

,pclass,survived,age,embarked,sex
0,2,1,29.0000,"(0, 0, 1)","(1, 0)"
1,2,0,2.0000,"(0, 0, 1)","(1, 0)"
2,2,0,30.0000,"(0, 0, 1)","(0, 1)"
3,2,0,25.0000,"(0, 0, 1)","(1, 0)"
4,2,1,0.9167,"(0, 0, 1)","(0, 1)"


This also can be done with a sklearn preprocessing method call *OneHotEncoder*

<br>
<br>

## Inverse Mapping

If we want to convert the features back into its original representation, we can simply do so my using inverted mapping dictionaries:

In [8]:
inv_label_mapping = {v: k for k, v in label_mapping.items()}
inv_pclass_mapping = {v: k for k, v in pclass_mapping.items()}
inv_embarked_mapping = {v: k for k, v in embarked_mapping.items()}
inv_sex_mapping = {v: k for k, v in sex_mapping.items()}

df['survived'] = df['survived'].map(inv_label_mapping)
df['pclass'] = df['pclass'].map(inv_pclass_mapping)
df['embarked'] = df['embarked'].map(inv_embarked_mapping)
df['sex'] = df['sex'].map(inv_sex_mapping)
df.head()

,pclass,survived,age,embarked,sex
0,1st,KO,29.0000,Southampton,female
1,1st,No KO,2.0000,Southampton,female
2,1st,No KO,30.0000,Southampton,male
3,1st,No KO,25.0000,Southampton,female
4,1st,KO,0.9167,Southampton,male


<br>
<br>

## Using scikit-learn and pandas features

The scikit-learn maching library comes with many useful preprocessing functions that we can use for our convenience.

<br>
<br>

### scikit LabelEncoder

In [9]:
from sklearn.preprocessing import LabelEncoder

class_le = LabelEncoder()
df['survived'] = class_le.fit_transform(df['survived'])

df.head()

,pclass,survived,age,embarked,sex
0,1st,0,29.0000,Southampton,female
1,1st,1,2.0000,Southampton,female
2,1st,1,30.0000,Southampton,male
3,1st,1,25.0000,Southampton,female
4,1st,0,0.9167,Southampton,male


The class labels can be converted back from integer to string via the `inverse_transform` method:

In [10]:
class_le.inverse_transform(df['survived'])

array(['KO', 'No KO', 'No KO', 'No KO', 'KO', 'KO', 'KO', 'No KO', 'KO',
       'No KO', 'No KO', 'KO', 'KO', 'KO', 'No KO', 'KO', 'No KO', 'No KO',
       'KO', 'KO', 'KO', 'No KO', 'KO', 'KO', 'KO', 'No KO', 'KO', 'KO',
       'KO', 'No KO', 'KO', 'KO', 'KO', 'No KO', 'No KO', 'No KO', 'KO',
       'KO', 'KO', 'No KO', 'KO', 'KO', 'KO', 'KO', 'No KO', 'No KO',
       'No KO', 'KO', 'KO', 'KO', 'KO', 'No KO', 'KO', 'No KO', 'KO',
       'No KO', 'KO', 'KO', 'KO', 'KO', 'KO', 'KO', 'No KO', 'KO', 'No KO',
       'No KO', 'KO', 'No KO', 'KO', 'KO', 'No KO', 'No KO', 'KO', 'KO',
       'No KO', 'KO', 'KO', 'KO', 'No KO', 'KO', 'KO', 'KO', 'KO', 'KO',
       'KO', 'KO', 'KO', 'No KO', 'KO', 'KO', 'KO', 'No KO', 'KO', 'KO',
       'No KO', 'KO', 'KO', 'No KO', 'KO', 'No KO', 'KO', 'KO', 'No KO',
       'KO', 'No KO', 'KO', 'KO', 'KO', 'KO', 'KO', 'KO', 'No KO', 'KO',
       'No KO', 'KO', 'KO', 'KO', 'KO', 'No KO', 'KO', 'No KO', 'KO', 'KO',
       'KO', 'KO', 'No KO', 'KO', 'KO', 'KO', 'K

<br>
<br>

### scikit  DictVectorizer

The [`DictVectorizer`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html) is another handy tool for feature extraction. The `DictVectorizer` takes a list of dictionary entries (feature-value mappings) and transforms it to vectors. The expected input looks like this:

In [11]:
df.head()

,pclass,survived,age,embarked,sex
0,1st,0,29.0000,Southampton,female
1,1st,1,2.0000,Southampton,female
2,1st,1,30.0000,Southampton,male
3,1st,1,25.0000,Southampton,female
4,1st,0,0.9167,Southampton,male


Note that the dictionary keys in each row represent the feature column labels. 

<br>
<br>

Now, we can use the `DictVectorizer` to turn this
mapping into a matrix:

In [12]:
from sklearn.feature_extraction import DictVectorizer
dvec = DictVectorizer(sparse=False)

X = dvec.fit_transform(df.transpose().to_dict().values())
X

array([[ 29.,   0.,   0., ...,   1.,   0.,   0.],
       [  2.,   0.,   0., ...,   1.,   0.,   1.],
       [ 30.,   0.,   0., ...,   0.,   1.,   1.],
       ..., 
       [ 37.,   0.,   0., ...,   0.,   1.,   1.],
       [ 28.,   0.,   0., ...,   0.,   1.,   1.],
       [ 19.,   0.,   0., ...,   0.,   1.,   1.]])

<br>
<br>

As we can see in the array above, the columns were reordered during the conversion (due to the hash mapping when we used the dictionary). However, we can simply add back the column names via the `get_feature_names` function.

In [13]:
df_new = pd.DataFrame(X, columns=dvec.get_feature_names())
df_new.head()

,age,embarked=Cherbourg,embarked=Queenstown,embarked=Southampton,pclass=1st,pclass=2nd,pclass=3rd,sex=female,sex=male,survived
0,29.0000,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
1,2.0000,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
2,30.0000,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0
3,25.0000,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
4,0.9167,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0


<br>
<br>

### scikit OneHotEncoder

Another useful tool in scikit-learn is the [`OneHotEncoder`](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html). The idea is the same as in the `DictVectorizer` example above; the only difference is that the `OneHotEncoder` takes integer columns as input. Here we are an `LabelEncoder`, we use the `LabelEncoder` first, to prepare the `color` column before we use the `OneHotEncoder`.

In [14]:
color_le = LabelEncoder()
df['embarked'] = color_le.fit_transform(df['embarked'])

df.head()

,pclass,survived,age,embarked,sex
0,1st,0,29.0000,2,female
1,1st,1,2.0000,2,female
2,1st,1,30.0000,2,male
3,1st,1,25.0000,2,female
4,1st,0,0.9167,2,male


In [15]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)

X = ohe.fit_transform(df[['embarked']].values)
X

array([[ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       ..., 
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.]])

<br>
<br>

### pandas get_dummies

Also, pandas comes with a convenience function to create new categories for nominal features, namely: [`get_dummies`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.reshape.get_dummies.html).
Using the get_dummies will create a new column for every unique string in a certain column:

In [16]:
df_new = pd.get_dummies(df)
df_new.head()

,survived,age,embarked,pclass_1st,pclass_2nd,pclass_3rd,sex_female,sex_male
0,0,29.0000,2,1,0,0,1,0
1,1,2.0000,2,1,0,0,1,0
2,1,30.0000,2,1,0,0,0,1
3,1,25.0000,2,1,0,0,1,0
4,0,0.9167,2,1,0,0,0,1


Note that the `get_dummies` function leaves the numeric columns untouched, how convenient!

In [18]:
df_new.to_csv('./data/dataset_newTitanic.csv', index=False)